In [1]:
from models import *
from utils import  *
import torch
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tqdm import tqdm

Files already downloaded and verified
Files already downloaded and verified


### Step 1: Obtain the student and teacher models which allow feature map extraction

In [25]:
VGG16_teacher = vgg16_bn_f(False,100)
VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))

<All keys matched successfully>

In [26]:
VGG11_student = vgg11_bn_f(False,100)

Distill using FitNets

In [9]:
fitnet = create_model(VGG16_teacher, VGG11_student, 1,0.75)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.08 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.04 Train Accuracy: 0.04
Epoch: 3 Train Loss: 1.01 Train Accuracy: 0.06
Epoch: 4 Train Loss: 0.97 Train Accuracy: 0.07
Epoch: 5 Train Loss: 0.94 Train Accuracy: 0.09
Epoch: 6 Train Loss: 0.91 Train Accuracy: 0.10
Epoch: 7 Train Loss: 0.89 Train Accuracy: 0.11
Epoch: 8 Train Loss: 0.87 Train Accuracy: 0.13
Epoch: 9 Train Loss: 0.85 Train Accuracy: 0.14
Epoch: 10 Train Loss: 0.83 Train Accuracy: 0.15
Epoch: 11 Train Loss: 0.81 Train Accuracy: 0.16
Epoch: 12 Train Loss: 0.80 Train Accuracy: 0.17
Epoch: 13 Train Loss: 0.78 Train Accuracy: 0.18
Epoch: 14 Train Loss: 0.77 Train Accuracy: 0.20
Epoch: 15 Train Loss: 0.75 Train Accuracy: 0.21
Epoch: 16 Train Loss: 0.74 Train Accuracy: 0.22
Epoch: 17 Train Loss: 0.72 Train Accuracy: 0.24
Epoch: 18 Train Loss: 0.71 Train Accuracy: 0.25
Epoch: 19 Train Loss: 0.69 Train Accuracy: 0.26
Epoch: 20 Train Loss: 0.68 Train Accuracy: 0.27
Epoch: 21 Train Loss: 0.67 Train Accuracy: 0.29
E

In [12]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.38217998
Final Test Accuracy: 0.3769
Final Train Loss: 0.5639470582804107
Final Test Loss: 2.356790871377204


In [13]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [15]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")

In [19]:
fitnet = create_model(VGG16_teacher, VGG11_student, 0,0.75)

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.11 Train Accuracy: 0.02
Epoch: 2 Train Loss: 1.06 Train Accuracy: 0.04
Epoch: 3 Train Loss: 1.03 Train Accuracy: 0.05
Epoch: 4 Train Loss: 1.00 Train Accuracy: 0.06
Epoch: 5 Train Loss: 0.97 Train Accuracy: 0.07
Epoch: 6 Train Loss: 0.95 Train Accuracy: 0.09
Epoch: 7 Train Loss: 0.93 Train Accuracy: 0.09
Epoch: 8 Train Loss: 0.91 Train Accuracy: 0.11
Epoch: 9 Train Loss: 0.89 Train Accuracy: 0.12
Epoch: 10 Train Loss: 0.87 Train Accuracy: 0.13
Epoch: 11 Train Loss: 0.85 Train Accuracy: 0.14
Epoch: 12 Train Loss: 0.84 Train Accuracy: 0.15
Epoch: 13 Train Loss: 0.82 Train Accuracy: 0.16
Epoch: 14 Train Loss: 0.81 Train Accuracy: 0.17
Epoch: 15 Train Loss: 0.79 Train Accuracy: 0.18
Epoch: 16 Train Loss: 0.78 Train Accuracy: 0.19
Epoch: 17 Train Loss: 0.76 Train Accuracy: 0.20
Epoch: 18 Train Loss: 0.75 Train Accuracy: 0.21
Epoch: 19 Train Loss: 0.73 Train Accuracy: 0.22
Epoch: 20 Train Loss: 0.72 Train Accuracy: 0.24
Epoch: 21 Train Loss: 0.71 Train Accuracy: 0.25
E

In [21]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.341
Final Test Accuracy: 0.33359998
Final Train Loss: 0.6087861097682162
Final Test Loss: 2.5102844435697906


In [22]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [23]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")

In [27]:
fitnet = create_model(VGG16_teacher, VGG11_student, 2,0.75)

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.09 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.04 Train Accuracy: 0.04
Epoch: 3 Train Loss: 1.01 Train Accuracy: 0.06
Epoch: 4 Train Loss: 0.98 Train Accuracy: 0.07
Epoch: 5 Train Loss: 0.95 Train Accuracy: 0.08
Epoch: 6 Train Loss: 0.92 Train Accuracy: 0.10
Epoch: 7 Train Loss: 0.89 Train Accuracy: 0.11
Epoch: 8 Train Loss: 0.87 Train Accuracy: 0.13
Epoch: 9 Train Loss: 0.85 Train Accuracy: 0.14
Epoch: 10 Train Loss: 0.83 Train Accuracy: 0.16
Epoch: 11 Train Loss: 0.80 Train Accuracy: 0.17
Epoch: 12 Train Loss: 0.78 Train Accuracy: 0.18
Epoch: 13 Train Loss: 0.77 Train Accuracy: 0.20
Epoch: 14 Train Loss: 0.75 Train Accuracy: 0.21
Epoch: 15 Train Loss: 0.74 Train Accuracy: 0.22
Epoch: 16 Train Loss: 0.72 Train Accuracy: 0.24
Epoch: 17 Train Loss: 0.70 Train Accuracy: 0.25
Epoch: 18 Train Loss: 0.68 Train Accuracy: 0.27
Epoch: 19 Train Loss: 0.67 Train Accuracy: 0.28
Epoch: 20 Train Loss: 0.66 Train Accuracy: 0.29
Epoch: 21 Train Loss: 0.64 Train Accuracy: 0.31
E

In [29]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.42069998
Final Test Accuracy: 0.4217
Final Train Loss: 0.5286962235980022
Final Test Loss: 2.1754487061956125


In [30]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [31]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")

In [ ]:
fitnet = create_model(VGG16_teacher, VGG11_student, 3,0.75)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.08 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.04 Train Accuracy: 0.04
Epoch: 3 Train Loss: 1.01 Train Accuracy: 0.06
Epoch: 4 Train Loss: 0.97 Train Accuracy: 0.07
Epoch: 5 Train Loss: 0.94 Train Accuracy: 0.09
Epoch: 6 Train Loss: 0.91 Train Accuracy: 0.10
Epoch: 7 Train Loss: 0.89 Train Accuracy: 0.11
Epoch: 8 Train Loss: 0.87 Train Accuracy: 0.13
Epoch: 9 Train Loss: 0.85 Train Accuracy: 0.14
Epoch: 10 Train Loss: 0.83 Train Accuracy: 0.15
Epoch: 11 Train Loss: 0.81 Train Accuracy: 0.16
Epoch: 12 Train Loss: 0.80 Train Accuracy: 0.17
Epoch: 13 Train Loss: 0.78 Train Accuracy: 0.18
Epoch: 14 Train Loss: 0.77 Train Accuracy: 0.20
Epoch: 15 Train Loss: 0.75 Train Accuracy: 0.21
Epoch: 16 Train Loss: 0.74 Train Accuracy: 0.22
Epoch: 17 Train Loss: 0.72 Train Accuracy: 0.24
Epoch: 18 Train Loss: 0.71 Train Accuracy: 0.25
Epoch: 19 Train Loss: 0.69 Train Accuracy: 0.26
Epoch: 20 Train Loss: 0.68 Train Accuracy: 0.27
Epoch: 21 Train Loss: 0.67 Train Accuracy: 0.29
E

In [ ]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.38217998
Final Test Accuracy: 0.3769
Final Train Loss: 0.5639470582804107
Final Test Loss: 2.356790871377204


In [ ]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [ ]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")